# Detectando glaciares en imágenes de satélite con ViT

## Librerías adicionales

Librerías no incluidas en el entorno de Google Colab

In [1]:
!pip install transformers[sentencepiece] datasets evaluate wandb -qqq

# Imports y Constantes

## Imports

A continuación se hacen los import de los módulos y librerías en Python para un proyecto de procesamiento de imágenes y modelado de lenguaje.
  La línea from datasets import load_dataset, load_from_disk, DatasetDict importa funciones específicas del módulo datasets para cargar y trabajar con datasets.
  La línea import evaluate importa un módulo llamado evaluate que se utiliza para evaluar el rendimiento de un modelo.
  La línea import torch importa la librería PyTorch, que es una plataforma popular para el aprendizaje profundo y el procesamiento de imágenes.
  Las líneas from transformers import (ViTFeatureExtractor, ViTForImageClassification, TrainingArguments, Trainer, EarlyStoppingCallback, pipeline) importan componentes específicos de la librería Transformers relacionados con la extracción de características, la clasificación de imágenes, los argumentos de entrenamiento, el entrenador, el controlador de detención temprana y la tubería.
  La línea from transformers import set_seed importa una función para establecer la semilla de números aleatorios en la librería Transformers.
  La línea import numpy as np importa la librería NumPy, que es una librería de cálculo numérico popular en Python.
  La línea import wandb importa la librería Weights & Biases (W&B), que es una plataforma de monitoreo y visualización de entrenamiento de modelos de aprendizaje automático.

In [2]:
# datasets
from datasets import load_dataset, load_from_disk, DatasetDict
import evaluate

# pytorch
import torch

# transformers
from transformers import (
    ViTFeatureExtractor, ViTForImageClassification,
    TrainingArguments, Trainer, EarlyStoppingCallback,
    pipeline
)
from transformers import set_seed
# arrays
import numpy as np

# wandb
import wandb

In [3]:
# fijar seed para inicializar pesos en la capa de clasificación siempre de la misma forma
set_seed(42)

In [4]:
# check if there is GPU available
!nvidia-smi

Tue Feb  7 17:22:03 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    15W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### wandb login

In [5]:
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: cgsellersm01. Use `wandb login --relogin` to force relogin


True

Con W&B, podemos utilizar variables de entorno para ciertas cosas. En este caso:
- Nombre del notebook
- Proyecto
- Registrar modelo automáticamente como artefacto

In [6]:
%env WANDB_NOTEBOOK_NAME=3_final_vit_glaciar_dataset_v2
#%env WANDB_PROJECT=NEW_vit-image-classification-glacier
%env WANDB_PROJECT=vit-clasificacion
# guardar modelo
%env WANDB_LOG_MODEL=true

env: WANDB_NOTEBOOK_NAME=3_final_vit_glaciar_dataset_v2
env: WANDB_PROJECT=vit-clasificacion
env: WANDB_LOG_MODEL=true


## Constantes

In [7]:
# dataset
HF_DATASET_IDENTIFIER = "alkzar90/rock-glacier-dataset"
HF_DATASET_CONFIG = "image-classification"

# model path
HF_MODEL_PATH = "google/vit-base-patch16-224-in21k"

# feature extractor from model checkpoint
FEATURE_EXTRACTOR = ViTFeatureExtractor.from_pretrained(HF_MODEL_PATH)

# metrics
ACCURACY = evaluate.load("accuracy")
FSCORE = evaluate.load("f1")

# early stopping
EARLY_STOPPING_CALLBACK = EarlyStoppingCallback(
    early_stopping_patience=5
)

# wandb
#WANDB_PROJECT_NAME = "NEW_vit-image-classification-glacier"
WANDB_PROJECT_NAME = "vit-clasificacion"
#WANDB_TEAM = "anban-dl"
WANDB_TEAM = "cgsellersm01"

/usr/local/lib/python3.8/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


# Descargar los datos

## Registrar un tabla en W&B para visualizar los datos

In [8]:
wandb.init(
    project=WANDB_PROJECT_NAME,
    entity=WANDB_TEAM,
    job_type="dataset-splits-logging"
)

# use artifact in wandb
# artifact name can be found in the artifact in W&B
artifact = wandb.use_artifact(
    #"anban-dl/NEW_vit-image-classification-glacier/raw_rock-glacier-dataset:v0"
    "cgsellersm01/vit-clasificacion/raw_rock-glacier-dataset:v0"
)
artifact_dir = artifact.download()
raw_datasets = load_from_disk(artifact_dir)

wandb: Downloading large artifact raw_rock-glacier-dataset:v0, 312.26MB. 10 files... 
wandb:   10 of 10 files downloaded.  
Done. 0:0:0.1


In [9]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['image', 'labels', 'path'],
        num_rows: 7875
    })
    validation: Dataset({
        features: ['image', 'labels', 'path'],
        num_rows: 1125
    })
    test: Dataset({
        features: ['image', 'labels', 'path'],
        num_rows: 2700
    })
})

In [10]:
def create_train_val_table(dataset: DatasetDict) -> wandb.data_types.Table:
    """Create a table with data in W&B for exploratory data analysis"""
    table = wandb.Table(columns=["path", "image", "split", "label"])

    for row in dataset["train"]:
        table.add_data(
            str(row["path"]),
            wandb.Image(row["image"]),
            "train",
            row["labels"]
        )

    for row in dataset["validation"]:
        table.add_data(
            str(row["path"]),
            wandb.Image(row["image"]),
            "validation",
            row["labels"]
        )

    return table

In [11]:
# create table
table = create_train_val_table(raw_datasets)

In [12]:
# start run and log table
run = wandb.init(
    project=WANDB_PROJECT_NAME,
    #entity=WANDB_TEAM,
    job_type="upload",
    name="add table",
    notes="add training and validation data table"
)
train_val_data = wandb.Artifact("table_train_val_data", type="table")
train_val_data.add(table, "train_val_data")
run.log_artifact(train_val_data)
run.finish()

# Pre-procesado de los datos

Utilizamos `ViTFeatureExtractor` para preprocesar los datos para ViT. Hugging Face lo tiene disponible en la librería `transformers`.

**IMPORTANTE** - Recordad utilizar la misma version de feature extractor que el modelo que vais a utilizar (google/vit-base-patch16-224-in21k)


Configuración for defecto para ViT.

In [15]:
FEATURE_EXTRACTOR

ViTFeatureExtractor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTFeatureExtractor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}

In [13]:
def transform(batch: dict) -> dict:
  """Transform images accoding to ViT requirements"""
  inputs = FEATURE_EXTRACTOR(
      [x.convert("RGB") for x in batch["image"]],
      return_tensors="pt"
  )
  inputs["labels"] = batch["labels"]
  return inputs

In [14]:
# apply transformations to dataset
preprocessed_dataset = raw_datasets.with_transform(transform)

In [15]:
# check
preprocessed_dataset["train"][0]

{'pixel_values': tensor([[[-0.4431, -0.4431, -0.4431,  ..., -0.4118, -0.4039, -0.3961],
          [-0.4431, -0.4431, -0.4431,  ..., -0.4118, -0.4039, -0.3961],
          [-0.4353, -0.4353, -0.4431,  ..., -0.4039, -0.4039, -0.4039],
          ...,
          [-0.3020, -0.3020, -0.3098,  ..., -0.2627, -0.2549, -0.2471],
          [-0.3098, -0.3098, -0.3098,  ..., -0.2549, -0.2392, -0.2314],
          [-0.3098, -0.3098, -0.3098,  ..., -0.2471, -0.2314, -0.2235]],
 
         [[-0.5373, -0.5373, -0.5373,  ..., -0.5294, -0.5216, -0.5137],
          [-0.5294, -0.5373, -0.5373,  ..., -0.5294, -0.5216, -0.5137],
          [-0.5216, -0.5294, -0.5373,  ..., -0.5294, -0.5216, -0.5216],
          ...,
          [-0.4588, -0.4588, -0.4588,  ..., -0.4039, -0.3961, -0.3961],
          [-0.4588, -0.4588, -0.4588,  ..., -0.3961, -0.3882, -0.3882],
          [-0.4588, -0.4588, -0.4588,  ..., -0.3961, -0.3882, -0.3882]],
 
         [[-0.5765, -0.5765, -0.5843,  ..., -0.5922, -0.5843, -0.5765],
          [-

# Entrenamiento y evaluación

## Utils

Funciones de utilidad que utilizaremos durante el entrenamiento.
   "collate_fn" toma como entrada un diccionario llamado "batch" y devuelve otro diccionario con dos claves: "pixel_values" y "labels". "pixel_values" es una pila (stack) de tensores de Torch formados por los valores de los pixeles de los elementos en "batch". "labels" es un tensor de Torch que contiene las etiquetas correspondientes a cada elemento en "batch".

   "compute_metrics" toma como entrada un objeto "preds" y devuelve un diccionario con dos claves: "accuracy" y "f1". "accuracy" es el valor de la precisión calculada mediante la función "ACCURACY.compute" y "f1" es el valor del F1 Score calculado mediante la función "FSCORE.compute". La precisión y el F1 Score se calculan comparando los índices de la clase más probable (calculados por np.argmax en "preds.predictions", eje=1) con las etiquetas reales (en "preds.label_ids").

In [16]:
def collate_fn(batch: dict) -> dict:
    """Function that processes a batch"""
    return {
        'pixel_values': torch.stack([torch.Tensor(x['pixel_values']) for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }


def compute_metrics(preds) -> dict:
    """Compute accuracy and f1 score"""
    accuracy = ACCURACY.compute(
        predictions=np.argmax(preds.predictions, axis=1),
        references=preds.label_ids
    )
    f_score = FSCORE.compute(
        predictions=np.argmax(preds.predictions, axis=1),
        references=preds.label_ids
    )
    return {
        "accuracy": accuracy,
        "f1": f_score
    }

## Fine-tune

Función para el entrenamiento
Toma como entrada una tabla de WandB, un diccionario de datos llamado "dataset", dos arrays de Numpy llamados "probs" y "preds", y una cadena "split". La función agrega información a la tabla de resumen, que luego se registrará en W&B.

Por cada fila en el diccionario "dataset[split]", la función extrae la ruta del archivo, la imagen, la etiqueta y las probabilidades y predicción correspondientes. Luego, la información se agrega a la tabla con el método "table.add_data". La información agregada incluye la ruta, la imagen, la etiqueta, la predicción y las probabilidades.

In [17]:
def add_to_summary_table(
    table: wandb.data_types.Table,
    dataset: DatasetDict,
    probs: np.array,
    preds: np.array,
    split: str
) -> None:
    """Create a summary table with predictions and labels to be logged to W&B"""
    for row_dict, probs, pred in zip(dataset[split], probs, preds):
        path = str(row_dict["path"])
        image = wandb.Image(row_dict["image"])
        label = row_dict["labels"]
        # add to table
        table.add_data(path, image, split, label, pred, probs)

A continuación, la función llamada "fine_tune" que realiza un ajuste fino en un modelo de clasificación de imágenes basado en el modelo ViT y evalúa el resultado en un conjunto de validación. La función recibe cuatro argumentos: "run_name", "run_notes", "tags" y "config". El objetivo final es entrenar un modelo y registrar los resultados en W&B (Weights & Biases).

La función inicializa una nueva ejecución (run) en W&B, descarga los conjuntos de datos necesarios y crea un modelo ViT con etiquetas específicas. Luego, se definen los argumentos de entrenamiento y se inicializa un entrenador con el modelo, los argumentos de entrenamiento y los datos preprocesados. Finalmente, se entrena el modelo y se registran las predicciones en los conjuntos de datos de entrenamiento y validación en W&B a través de una tabla resumen

In [20]:
def fine_tune(
    run_name: str,
    run_notes: str,
    tags: list,
    config: dict
) -> Trainer:
    """
    Fine-tune ViT and evaluate on validation set.
    Log fine-tune and model to W&B.
    """
    # initialize new run
    with wandb.init(
        project=WANDB_PROJECT_NAME,
        name=run_name,
        notes=run_notes,
        tags=tags
    ) as run:

        # download dataset from W&B
        #dataset = run.use_artifact("anban-dl/vit-image-classification-glacier/raw_rock-glacier-dataset:v0")
        dataset = run.use_artifact("cgsellersm01/vit-clasificacion/raw_rock-glacier-dataset:v0")
        data_dir = dataset.download()
        dataset = load_from_disk(data_dir)
        # extract label names
        labels = dataset['train'].features['labels'].names

        # download preprocessed datasets
        preprocessed_dataset = dict()
        for split in ["train", "validation"]:
            #split_dataset = run.use_artifact(f"anban-dl/vit-image-classification-glacier/{split}_preprocessed:v0")
            split_dataset = run.use_artifact(f"cgsellersm01/vit-clasificacion/{split}_preprocessed:v0")
            data_dir = split_dataset.download()
            split_dataset = load_from_disk(data_dir)
            preprocessed_dataset[split] = split_dataset

        # create model
        model = ViTForImageClassification.from_pretrained(
            HF_MODEL_PATH,
            num_labels=len(labels),
            id2label={str(i): c for i, c in enumerate(labels)},
            label2id={c: str(i) for i, c in enumerate(labels)}
        )

        # arguments
        training_args = TrainingArguments(
            output_dir="./vit-base-demo",
            per_device_train_batch_size=config["per_device_train_batch_size"], # IMP entrenamiento
            per_device_eval_batch_size=config["per_device_eval_batch_size"], # IMP entrenamiento
            save_strategy="steps",
            evaluation_strategy="steps",
            save_steps=50,
            eval_steps=50,
            num_train_epochs=config["num_train_epochs"], # IMP entrenamiento
            fp16=True,
            logging_strategy="steps",
            logging_steps=50,
            learning_rate=config["learning_rate"], # IMP entrenamiento
            weight_decay=config["weight_decay"], # IMP entrenamiento
            save_total_limit=2,
            remove_unused_columns=False,
            push_to_hub=False,
            load_best_model_at_end=True,
            report_to="wandb"
        )

        # default optim is AdamW
        trainer = Trainer(
            model=model,
            args=training_args,
            data_collator=collate_fn,
            compute_metrics=compute_metrics,
            train_dataset=preprocessed_dataset["train"],
            eval_dataset=preprocessed_dataset["validation"],
            tokenizer=FEATURE_EXTRACTOR,
            callbacks=[EARLY_STOPPING_CALLBACK]
        )

        # train
        trainer.train()

        # log predictions on train set and validation set
        train_probs= trainer.predict(
            preprocessed_dataset["train"]
        ).predictions
        train_preds = np.argmax(train_probs, axis=1)

        val_probs = trainer.predict(
            preprocessed_dataset["validation"]
        ).predictions
        val_preds = np.argmax(val_probs, axis=1)

        # summary table
        table = wandb.Table(
            columns=["path", "image", "split", "label", "prediction", "probs"]
        )
        
        # add train data
        add_to_summary_table(
            table,
            dataset,
            train_probs,
            train_preds,
            "train"
        )
        # add val data
        add_to_summary_table(
            table,
            dataset,
            val_probs,
            val_preds,
            "validation"
        )

        run.log({"summary_table": table})
        run.finish()

        return trainer

## Realizar un experimento
Se establecen tres variables: "run_name", "run_notes" y "tags". La variable "run_name" es un nombre de corrida (ejecución), "run_notes" es una nota o descripción del experimento y "tags" es una lista de etiquetas asociadas al experimento.

Luego, se definen los parámetros de entrenamiento en un diccionario llamado "config". Estos incluyen el tamaño del lote por dispositivo durante el entrenamiento y la evaluación, el número de épocas de entrenamiento, la tasa de aprendizaje y la tasa de decadencia del peso.

Finalmente, se llama a una función llamada "fine_tune" para realizar el ajuste fino. Se le pasan las tres variables establecidas anteriormente y el diccionario de configuración como argumentos.

In [ ]:
# experiment
run_name = "baseline"
run_notes = "primer experimento"
tags = ["baseline"]

# parámetros a definir para el entrenamiento
config = {
    "per_device_train_batch_size": 32,
    "per_device_eval_batch_size": 16,
    "num_train_epochs": 3,
    "learning_rate": 3e-5, # 0.001, 0.01
    "weight_decay": 0
}

# fine-tuning
trainer = fine_tune(
    run_name,
    run_notes,
    tags,
    config
)

wandb: Downloading large artifact raw_rock-glacier-dataset:v0, 312.26MB. 10 files... 
wandb:   10 of 10 files downloaded.  
Done. 0:0:0.0
wandb: Downloading large artifact train_preprocessed:v0, 4747.14MB. 12 files... 
wandb:   12 of 12 files downloaded.  
Done. 0:0:48.7
wandb: Downloading large artifact validation_preprocessed:v0, 677.78MB. 4 files... 
wandb:   4 of 4 files downloaded.  
Done. 0:0:6.3


Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Setting `WANDB_LOG_MODEL` from t

Step,Training Loss,Validation Loss,Accuracy,F1
50,0.470700,0.337021,{'accuracy': 0.8853333333333333},{'f1': 0.8577728776185226}
100,0.274700,0.223051,{'accuracy': 0.9191111111111111},{'f1': 0.9132507149666349}
150,0.217200,0.175713,{'accuracy': 0.9342222222222222},{'f1': 0.9273084479371315}
200,0.152800,0.210809,{'accuracy': 0.9182222222222223},{'f1': 0.9008620689655173}
250,0.149400,0.155684,{'accuracy': 0.9493333333333334},{'f1': 0.9400630914826499}
300,0.079700,0.108452,{'accuracy': 0.9653333333333334},{'f1': 0.9608040201005025}
350,0.079500,0.103136,{'accuracy': 0.9671111111111111},{'f1': 0.9629629629629629}
400,0.062300,0.094375,{'accuracy': 0.9706666666666667},{'f1': 0.9672943508424183}
450,0.062300,0.087944,{'accuracy': 0.9733333333333334},{'f1': 0.9698189134808852}
500,0.038400,0.101507,{'accuracy': 0.9724444444444444},{'f1': 0.9687814702920443}


***** Running Evaluation *****
  Num examples = 1125
  Batch size = 16
Saving model checkpoint to ./vit-base-demo/checkpoint-50
Configuration saved in ./vit-base-demo/checkpoint-50/config.json
Model weights saved in ./vit-base-demo/checkpoint-50/pytorch_model.bin
Image processor saved in ./vit-base-demo/checkpoint-50/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 1125
  Batch size = 16
Saving model checkpoint to ./vit-base-demo/checkpoint-100
Configuration saved in ./vit-base-demo/checkpoint-100/config.json
Model weights saved in ./vit-base-demo/checkpoint-100/pytorch_model.bin
Image processor saved in ./vit-base-demo/checkpoint-100/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 1125
  Batch size = 16
Saving model checkpoint to ./vit-base-demo/checkpoint-150
Configuration saved in ./vit-base-demo/checkpoint-150/config.json
Model weights saved in ./vit-base-demo/checkpoint-150/pytorch_model.bin
Image processor saved in ./vit-base-demo/

***** Running Prediction *****
  Num examples = 1125
  Batch size = 16


eval/loss,█▅▄▅▃▂▂▁▁▂▂▂▂▁
eval/runtime,█▂▁▂▆▄▆▆▄▂▅▅▇▁
eval/samples_per_second,▁▇█▇▃▅▃▃▅▇▄▄▂▇
eval/steps_per_second,▁▇█▇▃▄▃▃▅▇▄▄▂▇
train/epoch,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train/global_step,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇████
train/learning_rate,█▇▇▆▆▅▅▄▄▃▃▂▂▁
train/loss,█▅▄▃▃▂▂▂▂▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


# Evaluation on test set
Este código define dos funciones en Python:
  **transform_test**(batch: dict) -> dict: esta función toma como entrada un diccionario llamado "batch" que contiene imágenes y las transforma según los requisitos de ViT. Utiliza un extractor de características (FEATURE_EXTRACTOR) para convertir las imágenes en formato RGB y devuelve los tensores en formato PyTorch.

  **compute_metrics_inference**(preds: list, labels: list) -> dict: esta función toma como entrada dos listas "preds" y "labels" que representan las predicciones y las etiquetas respectivamente. Calcula la precisión y el puntaje F1 utilizando las funciones "ACCURACY.compute" y "FSCORE.compute", respectivamente, y devuelve un diccionario con las métricas "accuracy" y "f1".

In [ ]:
def transform_test(batch: dict) -> dict:
  """Transform images accoding to ViT requirements"""
  inputs = FEATURE_EXTRACTOR(
      [x.convert("RGB") for x in batch["image"]],
      return_tensors="pt"
  )
  return inputs

def compute_metrics_inference(preds: list, labels: list) -> dict:
    "Compute accuracy and F1 score"
    accuracy = ACCURACY.compute(
        predictions=preds,
        references=labels
    )
    f_score = FSCORE.compute(
        predictions=preds,
        references=labels
    )
    return {
        "accuracy": accuracy,
        "f1": f_score
    }

A continuación, tenemos un script que realiza la inferencia en un conjunto de prueba de una red neuronal de clasificación de imágenes previamente entrenada.

El script utiliza la biblioteca "Weights and Biases" (W&B) para llevar a cabo el proceso de inferencia.

El primer bloque de código crea un nuevo "run" (ejecución) en el proyecto W&B especificado por la constante WANDB_PROJECT_NAME y le da el nombre "inference_test_set".

Luego, conecta un artifact (modelo previamente entrenado) a este nuevo run, descargándolo de W&B.

Después, descarga un conjunto de prueba desde W&B y lo carga en memoria.

A continuación, crea un pipeline de inferencia usando la biblioteca "transformers" que se compone de tres elementos: un modelo previamente entrenado, un tokenizador (la función "transform_test") y una GPU.

El script hace una inferencia sobre cada imagen del conjunto de prueba y guarda las salidas en una lista.

Por último, el script calcula las métricas de rendimiento (accuracy y f1 score) usando las salidas y las etiquetas verdaderas del conjunto de prueba, y finaliza el run en W&B.

In [ ]:
# Create a new run
    # initialize new run
with wandb.init(
    project=WANDB_PROJECT_NAME,
    name="inference_test_set"
) as run:


    # Connect an Artifact to the run
    #best_model_name = "anban-dl/NEW_vit-image-classification-glacier/model-z81po1u1:v0"
    best_model_name = "jhbogasperona1970/vit-clasificacion/model-z81po1u1:v0"
    best_model_at = run.use_artifact(best_model_name)

    # Download model weights to a folder and return the path
    model_dir = best_model_at.download()

    # download dataset from W&B
    #test_dataset = run.use_artifact("anban-dl/NEW_vit-image-classification-glacier/rock-glacier-dataset_test:v0")
    test_dataset = run.use_artifact("jhbogasperona1970/vit-clasificacion/rock-glacier-dataset_test:v0")
    data_dir = test_dataset.download()
    test_dataset = load_from_disk(data_dir)
    # extract label names
    labels = test_dataset.features['labels'].names
    label2id = {c: str(i) for i, c in enumerate(labels)}

    # pipeline for inference
    pipe = pipeline(
        task='image-classification',
        model=model_dir,
        tokenizer=transform_test,
        device=0 # GPU
    )
    # get predictions
    outputs = []
    for output in pipe(test_dataset["image"]):
        outputs.append(output)
    
    preds = [label2id[out[0]["label"]] for out in outputs]
    # compute metrics
    metrics = compute_metrics_inference(preds, test_dataset["labels"])
    run.finish()

Necesitamos pasar un dict `{"pixel_values": tensor(---)}`

Aunque hayamos guardado los dataset preprocesador en W&B por reproducibilidad, tenemos que volver a transformar las imágenes.

Utilizamos una pipeline para realizar la inferencia de forma más cómoda y utilizando la GPU para acelerar los cálculos.

metrics es una variable en el código que almacena los resultados de la función compute_metrics_inference(). La función compute_metrics_inference toma dos listas de entrada, preds y labels, y devuelve un diccionario con las métricas computadas, que incluyen la precisión y el puntaje F1. Por lo tanto, la variable metrics contendrá un diccionario con las métricas calculadas de la inferencia en el modelo de clasificación de imágenes.

In [ ]:
metrics

Vamos que obtenemos alrededor de un 78% de accuracy y f1 score en el cojunto de testeo.

Si comparamos estos valores con las misma métricas de nuestro mejor modelo en el cojunto de validación, vemos que hay una diferencia considerable (97% en cojunto de validación). Esta diferencia quiere decir que nuestro modelo no generaliza tan bien cómo cabía esperar.

Esto puede ser debido a:
- El conjunto de validación es bastante más pequeño que el de testeo y puede que no sea representativo o esté desbalanceado (más imágenes de una clase que de otra). Deberíamos comprobar esto
- Nuestro modelo esté sobreajustado en los datos de entrenamiento y validación -> podríamos cambiar nuestro criterio de elección del mejor modelo y utlizar K-fold cross-validation solo con el training set